# Инструменты машинного обучения в Earth Engine 

В данном блокноте демонстрируются основные приемы работы с GEE на примере решения задачи средствами Earth Engine.

## Инициализация

Выполняетя по необходимости, в случае если не была выполнена ранее или в случае возникновения тайм-аута.

In [10]:
import ee
from IPython.display import Image

# Check if the server is authenticated. If not, display instructions that
# explain how to complete the process.
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException:
  print('The Earth Engine package failed to initialize!')
  !earthengine authenticate


The Earth Engine package initialized successfully!


In [3]:
!earthengine asset info users/kolesovdm/DVFU

{
  "id": "users/kolesovdm/DVFU",
  "type": "Folder"
}


In [4]:
!earthengine ls -l users/kolesovdm/DVFU

[Table]  users/kolesovdm/DVFU/simpleDataDVFU


## Обработка данных средствами Python API для Google Earth Engine. 

### Базовый пример

После инициализации становятся доступны функции Python API. Прочитаем, например содержимое таблицы `users/kolesovdm/DVFU/simpleDataDVFU`:

In [5]:
sample = ee.FeatureCollection('users/kolesovdm/DVFU/simpleDataDVFU')

Обратим внимание, что объект `sample` не содержит сами данные, на самом деле он связан с данными, лежащими на серверах Google Earth Engine. Все манипуляции с объектами Earth Engine производятся удаленно, поскольку код, использующий Python API для Earth Engine при выполнении передается на сервера Google, где компилируется во внутренее представление, а затем выполняется. 

In [6]:
print(sample)

ee.FeatureCollection({
  "type": "Invocation", 
  "arguments": {
    "tableId": "users/kolesovdm/DVFU/simpleDataDVFU"
  }, 
  "functionName": "Collection.loadTable"
})


Тем не менее есть возможность получить эти данные локально при помощи функции `.getInfo()`, но поступать так следует с осторожностью, т.к. передача данных по сети и/или выполнение программного кода локально очень замедляет процесс обработки.

In [7]:
# sample.getInfo()

### Обзор основных приемов работы с GEE

#### Доступные для анализа данные

Спутниковая съемка:
 * [Landsat](https://developers.google.com/earth-engine/datasets/catalog/landsat/)
 * [Sentinel](https://developers.google.com/earth-engine/datasets/catalog/sentinel/)
 * [MODIS](https://developers.google.com/earth-engine/datasets/catalog/modis)
 
 
[Аэрофотосъемка и съемка высокого разрешения](https://developers.google.com/earth-engine/datasets/tags/highres)

Климатические данные:
 * [Температуры](https://developers.google.com/earth-engine/datasets/tags/temperature)
 * [Общеклиматические данные](https://developers.google.com/earth-engine/datasets/tags/climate)
 * [Метеорологические данные](https://developers.google.com/earth-engine/datasets/tags/weather)
 * [Параметры атмосферы](https://developers.google.com/earth-engine/datasets/tags/atmosphere)
 

Геофизические данные:
 * [Высоты (цифровые модели местности)](https://developers.google.com/earth-engine/datasets/tags/elevation)
 * [Классы земного покрова (такие как лес, луг, вода, ...)](https://developers.google.com/earth-engine/datasets/tags/landcover)
 
 
Пользователь может добавлять/импортировать собственные данные (ASSETS).

##### Пример доступа к данным

У каждой коллекции данных есть свой идентификатор, по которому можно ее получить. Например, Sentinel-2 доступен по ссылке с идентификатором `COPERNICUS/S2`:

In [8]:
sentinel = ee.ImageCollection("COPERNICUS/S2")
print(sentinel)

ee.ImageCollection({
  "type": "Invocation", 
  "arguments": {
    "id": "COPERNICUS/S2"
  }, 
  "functionName": "ImageCollection.load"
})


Аналогично, у каждого изображения также есть собственный идентификатор, по которому можно сослаться на это изображение:

In [46]:
# Получим ссылку на изображение
my_img = ee.Image('COPERNICUS/S2/20181002T020651_20181002T021217_T52TGN');

# Отрисуем его
Image(url=my_img.getThumbUrl({'min': 0, 'max': 1500, 'bands': 'B7,B5,B3'}))

In [83]:
region = ee.Geometry.Rectangle([131.74, 42.95, 131.94, 43.07])
my_img = my_img = ee.Image('COPERNICUS/S2/20181002T020651_20181002T021217_T52TGN').clip(region)
Image(url=my_img.getThumbUrl({'min': 0, 'max': 2000, 'bands': 'B2,B5,B3'}))

#### Обзор API

##### Основные структуры данных

Два основных типа данных в GEE:
 * Image: растровый объект, состоит из каналов изображения и словаря свойств.
 * Feature: векоторный объект, состоит из геометрии (тип `Geometry`) и словаря свойств.
 
Производные от данных типов, например:
 * ImageCollection: стек изображений (растров).
 * FeatureCollection: список отдельных векторных объектов.

Аналогичные стандартным, знакомым по другим языкам (строки, числа и т.п.) типам:
 * Dictionary: словарь.
 * List: список.
 * Array: массив.
 * Geometry: геометрия.
 * Date: дата. 
 * Number: числовой тип.
 * String: строки.
У многих этих типов есть подтипы  (например, у числового - byte, float64 и т.п.).

Пример: отобразим свойства изображения.

In [78]:
prop = my_img.getInfo()
print(prop.keys())
# print(prop['properties'])
# print(prop['properties']['MEAN_INCIDENCE_ZENITH_ANGLE_B10']) # Зенитный угол Солнца в момент съемки

[u'bands', u'version', u'type', u'id', u'properties']


##### Основные алгоритмы

**Все перечисленные выше объекты являются объектами, хранящимися и обрабатываемыми на серверах Google.** Код, который вы пишите, отправляется на сервер, там компилируется и запускается. Объекты не обрабатываются локально.

GEE поддерживает множество встроенных алгоритмов обработки (гео)данных. Но нас будут интересовать, в первую очередь, алгоритмы машинного обучения:
 * Алогоритмы классификации:
  - Деревья решений.
  - Случайный лес.
  - Машины опорных векторов.
  - Байесовский классификатор.
  - ....
 * Алгоритмы кластеризации (в основном модификации К-средних).
 * Статистические методы:
  - Регрессии (классическая линейная, взвешенная, робастная, ...).
  - Статистические метрики качества классификаторов и сходства выборок.
 * ...
  